# Inteligencia Artificial Aplicada a Negocios y Empresas
# Parte 1 - Optimizacion de los flujos de trabajo en un almacen con Q-Learning


## Importacion de las librerias


In [12]:
import numpy as np

## Configuracion de los parametros gamma y alpha para el algoritmo de Q-Learning


In [13]:
#factor de descuento, este tiene más impacto en el algoritmo. 
gamma = 0.75
#El alpha tiene más que ver con el tiempo que tarda en converger. Si es un numero muy alto puede converger en un mínimo local
#el alpha indica como de rápido debe de incorporarse el aprendizaje. 
alpha = 0.9

# PARTE 1 - DEFINICION DEL ENTORNO


## Definicion de los estados


In [16]:
#Traduccion de estados a números
location_to_state = {'A': 0,
                     'B': 1,
                     'C': 2,
                     'D': 3,
                     'E': 4,
                     'F': 5,
                     'G': 6, 
                     'H': 7, 
                     'I': 8,
                     'J': 9,
                     'K': 10,
                     'L': 11}

In [17]:
#Prioridad de estados
prioridad = {1: 'G',
                     2: 'K',
                     3: 'L',
                     4: 'J',
                     5: 'A',
                     6: 'I',
                     7: 'H', 
                     8: 'C', 
                     9: 'B',
                     10: 'D',
                     11: 'F',
                     12: 'E'}

#Primera solución 
#Si hay un punto intermedio la recompensa debería ser suculenta.

#Segunda solución 
#si hay un punto intermedio por el que no queremos pasar le datos una recompensa negativa--> Dar refuerzo negativo a los puntos que no queremos

#La tercera solución 
#si hay un punto interesante por el que quiero pasar pues obligar a pasar por ahi, es decir, dividir la ruta en dos partes una desde el punto inicial
#a este punto y otro desde este punto al final. 

## Definicion de las acciones

In [18]:
actions = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11]

## Definicion de las recompensas
## Columnas:    A,B,C,D,E,F,G,H,I,J,K,L

In [20]:
#Define en la matriz los puntos del laberinto como están conectados
R = np.array([[0,1,0,0,0,0,0,0,0,0,0,0], # A
              [1,0,1,0,0,1,0,0,0,0,0,0], # B
              [0,1,0,0,0,0,1,0,0,0,0,0], # C
              [0,0,0,0,0,0,0,1,0,0,0,0], # D
              [0,0,0,0,0,0,0,0,1,0,0,0], # E
              [0,1,0,0,0,0,0,0,0,1,0,0], # F
              [0,0,1,0,0,0,1,1,0,0,0,0], # G
              [0,0,0,1,0,0,1,0,0,0,0,1], # H
              [0,0,0,0,1,0,0,0,0,1,0,0], # I
              [0,0,0,0,0,1,0,0,1,0,1,0], # J
              [0,0,0,0,0,0,0,0,0,1,0,1], # K
              [0,0,0,0,0,0,0,1,0,0,1,0]])# L

# PARTE 2 - CONSTRUCCION DE LA SOLUCION DE IA CON Q-LEARNING


## Transformacion inversa de estados a ubicaciones

In [24]:
state_to_location = {state : location for location, state in location_to_state.items()}

## Crear la funcion final que nos devuelva la ruta óptima

In [30]:
def route(starting_location, ending_location):
    R_new = np.copy(R)
    ending_state = location_to_state[ending_location]
    #Damos un peso muy grande al valor final --> Para que se vaya para allá
    R_new[ending_state, ending_state] = 1000
    #Inicialización de los valores Q donde se guardan los estados
    Q = np.array(np.zeros([12, 12]))
    #Vamos a realizar 1000 iteraciones para buscar la solución
    for i in range(1000):
        #Cojemos un estado aleatorio de los posibles (que son 12)
        current_state = np.random.randint(0, 12)
        #Aqui vamos a almacenar las acciones posibles desde el estado actual
        playable_actions = []
        for j in range(12):
            #Vamos a estar en un estado, o sea una fila de la matriz de acciones posibles
            #Vamos a recorrer esa fila viendo cuales son las acciones posibles
            if R_new[current_state, j] > 0:
                playable_actions.append(j)
        #Una vez buscadas las acciones posibles desde ese estado. Elegimos una de forma aleatoria        
        next_state = np.random.choice(playable_actions)
        #Implementamos el TD de la ecuación de Belman, next_state (es el estado actual porque el proximo estado calculado coincide con la acción actual)
        TD = R_new[current_state, next_state] + gamma*Q[next_state, np.argmax(Q[next_state,])] - Q[current_state, next_state]
        #finalmente actualizamos la matriz Q con lo que se ha aprendido. la nuevo Q es la anterior más el alfa por la diferencia temporal (TD)
        Q[current_state, next_state] = Q[current_state, next_state] + alpha*TD
    
    route = [starting_location]
    next_location = starting_location
    while(next_location != ending_location):
        #Traducimos de location a estate para ver en el estado que estamos
        starting_state = location_to_state[starting_location]
        #Buscamos para esa fila (estado el mejor estado)
        next_state = np.argmax(Q[starting_state, ])
        #Volvemos a traducir a la location
        next_location = state_to_location[next_state]
        route.append(next_location)
        #Actualizamos la localizacion del robot para la proxima itereacion.
        starting_location = next_location
    return route

In [26]:
def route_intermediary(starting_location, intermediary_location, ending_location):
    R_new = np.copy(R)
    ending_state = location_to_state[ending_location]
    intermediary_state = location_to_state[intermediary_location]
    #Damos un peso muy grande al valor final --> Para que se vaya para allá
    R_new[ending_state, ending_state] = 1000
    R_new[intermediary_state, intermediary_state] = 200
    #Inicialización de los valores Q donde se guardan los estados
    Q = np.array(np.zeros([12, 12]))
    #Vamos a realizar 1000 iteraciones para buscar la solución
    for i in range(1000):
        #Cojemos un estado aleatorio de los posibles (que son 12)
        current_state = np.random.randint(0, 12)
        #Aqui vamos a almacenar las acciones posibles desde el estado actual
        playable_actions = []
        for j in range(12):
            #Vamos a estar en un estado, o sea una fila de la matriz de acciones posibles
            #Vamos a recorrer esa fila viendo cuales son las acciones posibles
            if R_new[current_state, j] > 0:
                playable_actions.append(j)
        #Una vez buscadas las acciones posibles desde ese estado. Elegimos una de forma aleatoria        
        next_state = np.random.choice(playable_actions)
        #Implementamos el TD de la ecuación de Belman, next_state (es el estado actual porque el proximo estado calculado coincide con la acción actual)
        TD = R_new[current_state, next_state] + gamma*Q[next_state, np.argmax(Q[next_state,])] - Q[current_state, next_state]
        #finalmente actualizamos la matriz Q con lo que se ha aprendido. la nuevo Q es la anterior más el alfa por la diferencia temporal (TD)
        Q[current_state, next_state] = Q[current_state, next_state] + alpha*TD
    
    route = [starting_location]
    next_location = starting_location
    while(next_location != ending_location):
        #Traducimos de location a estate para ver en el estado que estamos
        starting_state = location_to_state[starting_location]
        #Buscamos para esa fila (estado el mejor estado)
        next_state = np.argmax(Q[starting_state, ])
        #Volvemos a traducir a la location
        next_location = state_to_location[next_state]
        route.append(next_location)
        #Actualizamos la localizacion del robot para la proxima itereacion.
        starting_location = next_location
    return route

# PARTE 3 - PONER EL MODELO EN PRODUCCION


In [31]:
def best_route(starting_location, intermediary_location, ending_location):
    return route(starting_location, intermediary_location) + route(intermediary_location, ending_location)[1:]

# Imprimir la ruta final
print("Ruta Elegida:")
print(best_route('E', 'B', 'G'))

Ruta Elegida:
['E', 'I', 'J', 'F', 'B', 'C', 'G']


In [29]:
ruta=route_intermediary('E', 'B', 'G')

KeyboardInterrupt: 